## Deliverable 2. Create a Customer Travel Destinations Map.

In [97]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [98]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pontal Do Parana,BR,-25.6533,-48.5128,72.99,56,0,6.26,clear sky
1,1,Rikitea,PF,-23.1203,-134.9692,71.55,85,100,5.35,moderate rain
2,2,Katsuura,JP,35.1333,140.3000,66.88,78,100,13.91,overcast clouds
3,3,Margate,GB,51.3813,1.3862,62.69,73,54,4.00,broken clouds
4,4,Bluff,NZ,-46.6000,168.3333,45.81,74,85,32.73,overcast clouds


In [99]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [100]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Oranjestad,AW,12.5240,-70.0270,85.95,70,40,24.16,scattered clouds
8,8,Namatanai,PG,-3.6667,152.4333,78.67,80,31,5.01,scattered clouds
10,10,Vaini,TO,-21.2000,-175.2000,75.88,75,99,11.16,overcast clouds
14,14,Maxixe,MZ,-23.8597,35.3472,78.62,89,75,16.11,broken clouds
20,20,Albany,US,42.6001,-73.9662,78.73,55,38,4.00,scattered clouds
21,21,Butaritari,KI,3.0707,172.7902,81.30,75,85,12.46,overcast clouds
26,26,Latung,PH,5.5000,120.8833,83.05,76,82,4.99,broken clouds
27,27,Hilo,US,19.7297,-155.0900,86.29,85,100,3.44,overcast clouds
30,30,Bensonville,LR,6.4461,-10.6125,80.20,86,100,6.20,light rain
31,31,Korla,CN,41.7597,86.1469,78.64,21,100,14.03,overcast clouds


In [101]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                246
City                   246
Country                246
Lat                    246
Lng                    246
Max Temp               246
Humidity               246
Cloudiness             246
Wind Speed             246
Current Description    246
dtype: int64

In [102]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head() 

246


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Oranjestad,AW,12.5240,-70.0270,85.95,70,40,24.16,scattered clouds
8,8,Namatanai,PG,-3.6667,152.4333,78.67,80,31,5.01,scattered clouds
10,10,Vaini,TO,-21.2000,-175.2000,75.88,75,99,11.16,overcast clouds
14,14,Maxixe,MZ,-23.8597,35.3472,78.62,89,75,16.11,broken clouds
20,20,Albany,US,42.6001,-73.9662,78.73,55,38,4.00,scattered clouds


In [103]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = None
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Oranjestad,AW,85.95,scattered clouds,12.5240,-70.0270,None
8,Namatanai,PG,78.67,scattered clouds,-3.6667,152.4333,None
10,Vaini,TO,75.88,overcast clouds,-21.2000,-175.2000,None
14,Maxixe,MZ,78.62,broken clouds,-23.8597,35.3472,None
20,Albany,US,78.73,scattered clouds,42.6001,-73.9662,None
21,Butaritari,KI,81.30,overcast clouds,3.0707,172.7902,None
26,Latung,PH,83.05,broken clouds,5.5000,120.8833,None
27,Hilo,US,86.29,overcast clouds,19.7297,-155.0900,None
30,Bensonville,LR,80.20,light rain,6.4461,-10.6125,None
31,Korla,CN,78.64,overcast clouds,41.7597,86.1469,None


In [104]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
      

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [105]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10) 

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Oranjestad,AW,85.95,scattered clouds,12.5240,-70.0270,Divi Village Golf & Beach Resort
8,Namatanai,PG,78.67,scattered clouds,-3.6667,152.4333,Seagulls Inn Namatani
10,Vaini,TO,75.88,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
14,Maxixe,MZ,78.62,broken clouds,-23.8597,35.3472,Oceano
21,Butaritari,KI,81.30,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
27,Hilo,US,86.29,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
30,Bensonville,LR,80.20,light rain,6.4461,-10.6125,Moinga
31,Korla,CN,78.64,overcast clouds,41.7597,86.1469,Gulisitan Hotel
37,Itarema,BR,84.13,overcast clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
40,Saint-Francois,FR,83.68,overcast clouds,46.4154,3.9054,Manoir Du Bonheur


In [106]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [107]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [108]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))